In [1]:
import os
import time
from datetime import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ivolatility as ivol

from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as dt

from nelson_siegel_svensson.calibrate import calibrate_nss_ols

# Models definitions

## Heston Model

In [62]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    """
        S0: initial asset price
        v0: initial variance
        kappa: mean reversion rate
        theta: long-term average volatility
        sigma: volatility of volatility
        rho: correlation between returns and variances under risk-neutral dynamics
        lambd: ?
        tau: ?
        r: ?
    """
    # constants
    a = kappa*theta
    b = kappa+lambd

    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j

    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )

    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)

    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)

    return exp1 * term2 * exp2


def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    numerator = np.exp(r * tau) * heston_charfunc(phi - 1j, *args) - K * heston_charfunc(phi, *args)
    denominator = 1j * phi * K ** (1j * phi) 
    return numerator/denominator


def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

    P, umax, N = 0, 100, 10000
    dphi = umax / N #dphi is width

    for i in range(1, N):
        # rectangular integration
        phi = dphi * (2*i + 1)/2 # midpoint to calculate height
        numerator = np.exp(r*tau) * heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)

        P += dphi * numerator/denominator

    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)


def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

    real_integral, err = np.real( quad(integrand, 0, 100, args=args) )

    return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

## Merton Jump Model

In [59]:
# --- Part 1: Merton Characteristic Function ---
def merton_charfunc(phi, S0, sigma, lambd, mu_j, sigma_j, tau, r):
    """
    Characteristic function for the Merton Jump Diffusion Model.
    """
    drift = r - 0.5 * sigma**2 - lambd * (np.exp(mu_j + 0.5 * sigma_j**2) - 1)
    exp1 = np.exp(1j * phi * (np.log(S0) + drift * tau))
    diffusion = -0.5 * sigma**2 * phi**2 * tau
    jumps = lambd * tau * (np.exp(1j * phi * mu_j - 0.5 * sigma_j**2 * phi**2) - 1)
    return exp1 * np.exp(diffusion + jumps)

# --- Part 2: Integrand for Numerical Integration ---
def integrand_merton(phi, S0, sigma, lambd, mu_j, sigma_j, tau, r, K):
    """
    Integrand for numerical integration in the Merton Jump Diffusion Model.
    """
    args = (S0, sigma, lambd, mu_j, sigma_j, tau, r)
    numerator = np.exp(r * tau) * merton_charfunc(phi - 1j, *args) - K * merton_charfunc(phi, *args)
    denominator = 1j * phi * K**(1j * phi)
    return numerator / denominator

# --- Part 3: Rectangular Numerical Integration ---
def merton_price_rec(S0, K, sigma, lambd, mu_j, sigma_j, tau, r):
    """
    Option price using rectangular integration for the Merton Jump Diffusion Model.
    """
    args = (S0, sigma, lambd, mu_j, sigma_j, tau, r)
    P, umax, N = 0, 100, 10000
    dphi = umax / N  # dphi is width

    for i in range(1, N):
        phi = dphi * (2 * i + 1) / 2  # midpoint to calculate height
        numerator = np.exp(r * tau) * merton_charfunc(phi - 1j, *args) - K * merton_charfunc(phi, *args)
        denominator = 1j * phi * K**(1j * phi)
        P += dphi * numerator / denominator

    return np.real((S0 - K * np.exp(-r * tau)) / 2 + P / np.pi)

# --- Part 4: Option Price Using Scipy Quad Integration ---
def merton_price(S0, K, sigma, lambd, mu_j, sigma_j, tau, r):
    """
    Option price using scipy quad integration for the Merton Jump Diffusion Model.
    """
    real_integral, err = np.real(quad(integrand_merton, 0, 100, args=(S0, sigma, lambd, mu_j, sigma_j, tau, r, K)))
    return (S0 - K * np.exp(-r * tau)) / 2 + real_integral / np.pi


## Kou Model

In [60]:
# --- Step 1: Kou Characteristic Function ---
def kou_charfunc(phi, S0, sigma, lambd, p, eta1, eta2, tau, r):
    """
    Characteristic function for the Kou Double Exponential Jump Model.
    """
    drift = r - 0.5 * sigma**2 - lambd * (p * eta1 / (eta1 + 1) + (1 - p) * eta2 / (eta2 + 1) - 1)
    exp1 = np.exp(1j * phi * (np.log(S0) + drift * tau))
    diffusion = -0.5 * sigma**2 * phi**2 * tau
    jumps = lambd * tau * (
        p * eta1 / (eta1 - 1j * phi) + (1 - p) * eta2 / (eta2 + 1j * phi) - 1
    )
    return exp1 * np.exp(diffusion + jumps)

# --- Step 2: Integrand for Numerical Integration ---
def integrand_kou(phi, S0, sigma, lambd, p, eta1, eta2, tau, r, K):
    """
    Integrand for numerical integration in the Kou model.
    """
    args = (S0, sigma, lambd, p, eta1, eta2, tau, r)
    numerator = np.exp(r * tau) * kou_charfunc(phi - 1j, *args) - K * kou_charfunc(phi, *args)
    denominator = 1j * phi * K**(1j * phi)
    return numerator / denominator

# --- Step 3: Rectangular Numerical Integration ---
def kou_price_rec(S0, K, sigma, lambd, p, eta1, eta2, tau, r):
    """
    Option price using rectangular integration for the Kou model.
    """
    args = (S0, sigma, lambd, p, eta1, eta2, tau, r)
    P, umax, N = 0, 100, 10000
    dphi = umax / N  # dphi is width

    for i in range(1, N):
        phi = dphi * (2 * i + 1) / 2  # midpoint to calculate height
        numerator = np.exp(r * tau) * kou_charfunc(phi - 1j, *args) - K * kou_charfunc(phi, *args)
        denominator = 1j * phi * K**(1j * phi)
        P += dphi * numerator / denominator

    return np.real((S0 - K * np.exp(-r * tau)) / 2 + P / np.pi)

# --- Step 4: Option Price Using Scipy Quad Integration ---
def kou_price(S0, K, sigma, lambd, p, eta1, eta2, tau, r):
    """
    Option price using scipy quad integration for the Kou model.
    """
    real_integral, err = np.real(quad(integrand_kou, 0, 100, args=(S0, sigma, lambd, p, eta1, eta2, tau, r, K)))
    return (S0 - K * np.exp(-r * tau)) / 2 + real_integral / np.pi

## Variance Gamma Model

In [61]:
def vg_charfunc(phi, S0, theta, sigma, nu, tau, r):
    """
    Variance Gamma characteristic function of ln(S_tau).
    
    ----------
    Parameters
    phi   : float or ndarray
        Frequency argument of the characteristic function.
    S0    : float
        Spot price at t=0.
    theta : float
        'Drift' parameter of the VG subordinator (sometimes denoted as θ).
    sigma : float
        'Vol' parameter of the VG subordinator.
    nu    : float
        'Gamma' parameter (sometimes 1/kappa in other notations).
    tau   : float
        Time to maturity (T - 0).
    r     : float
        Risk-free interest rate.
    -------
    Notes
    The key difference from Heston is the explicit "martingale adjustment" ω:
        ω = - (1/nu) * ln(1 - theta*nu - 0.5*sigma^2 * nu)
    so that discounted S_t is a martingale under the risk-neutral measure.
    -----
    """
    # Martingale adjustment: ensure E[e^{-r tau} * S_tau] = S_0
    omega = -(1.0 / nu) * np.log(1.0 - theta*nu - 0.5*sigma**2 * nu)
    
    # VG characteristic function for ln(S_tau):
    #   exp{ i φ [ ln(S0) + (r + ω)*τ ] } * (1 - i θ ν φ + 0.5 σ^2 ν φ^2)^(-τ/ν)
    return np.exp(
        1j * phi * (np.log(S0) + (r + omega) * tau)
    ) * (
        (1.0 - 1j * theta * nu * phi + 0.5 * (sigma ** 2) * nu * (phi ** 2)) ** (-tau / nu)
    )


def integrand_vg(phi, S0, K, theta, sigma, nu, tau, r):
    """
    Parameters
    phi   : float
        Integration variable (Fourier frequency).
    S0, K : float
        Spot (S0) and strike (K).
    theta, sigma, nu, tau, r : float
        VG parameters and maturity/rate.
    """
    numerator = (
        np.exp(1j * phi * r * tau) * vg_charfunc(phi - 1j, S0, theta, sigma, nu, tau, r)
        - K * vg_charfunc(phi, S0, theta, sigma, nu, tau, r)
    )
    denominator = 1j * phi * (1j * phi + 1.0)
    
    return numerator / denominator


def vg_price_rec(S0, K, theta, sigma, nu, tau, r):
    """
    Parameters
    S0, K : float
        Spot and strike.
    theta, sigma, nu : float
        VG parameters.
    tau   : float
        Time to maturity.
    r     : float
        Risk-free interest rate.
    """
   
    phi_max = 100.0
    N = 10000
    dphi = phi_max / N
    
    P = 0.0
    for i in range(1, N):
        # midpoint in each subinterval (rectangular rule)
        phi = dphi * (2 * i - 1) / 2.0
        P += dphi * integrand_vg(phi, S0, K, theta, sigma, nu, tau, r)
    
    return np.real( (S0 - K * np.exp(-r * tau)) / 2.0 + P / np.pi )

# Set up and Get Data

In [6]:
username = 'PutYourIVolatilityUSernameHere'
password = 'PutYourIVolatilityPasswordHere'

In [7]:
# loginto the IVol account
ivol.setLoginParams(username=username, password=password)

In [8]:
# Define the parameters for the options
# If the relative data is already present in the data folder, load it from there instead of querying the IVol API
symbol =            'UCG'
tradeDate =         '2024-10-04'
last_trade_date =   '2024-10-09'
region =            'EUROPE'
dteFrom =           10
dteTo =             1000
cp =                'C'
deltaFrom =         0
deltaTo =           1

In [ ]:
data_path = f'../data/{symbol}_{region}_{tradeDate}_dteFrom{dteFrom}-To{dteTo}_cp{cp}_deltaFrom{deltaFrom}-To{deltaTo}.csv'
data_path

In [ ]:
if not os.path.exists(data_path):
    getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')
    marketData = getMarketData(
        symbol=symbol, 
        tradeDate=tradeDate, 
        region=region,
        cp='C', 
        dteFrom=dteFrom, dteTo=dteTo, 
        deltaFrom=deltaFrom, deltaTo=deltaTo
    )
    marketData.to_csv(data_path)
else:
    marketData = pd.read_csv(data_path)
marketData

In [ ]:
marketData.columns

## Compute strikes and maturities

In [12]:
def calc_maturity(date):
    #invece che today mettere a data desiderata vecchia, per non avere maturities negative
    return ((dt.strptime(date, '%Y-%m-%d %H:%M:%S.%f') - dt.today()).days / 365.25)

In [13]:
all_strikes = [date_group['price_strike'].tolist() for date, date_group in marketData.groupby('expiration_date')]
common_strikes = sorted(set.intersection(*map(set, all_strikes)))

In [ ]:
num_exp_days = marketData['expiration_date'].unique().shape[0]
print(f"Number of expiration days: {num_exp_days} - number of common strikes: {len(common_strikes)}")

In [ ]:
maturities = []
all_prices = []

for date, date_group in marketData.groupby('expiration_date'):
    maturities.append(calc_maturity(date))
    # prices = date_group[date_group['price_strike'].isin(common_strikes)]['price'].tolist()
    prices = [date_group['price'].tolist()[i] for i,x in enumerate(date_group['price_strike'].tolist()) if x in common_strikes][:len(common_strikes)]
    all_prices.append(prices)

price_arr = np.array(all_prices)
price_arr.shape

## Compute volatility surface

In [ ]:
vol_surface = pd.DataFrame(price_arr, index=maturities, columns=common_strikes)
vol_surface

In [ ]:
vol_surface_long = vol_surface.melt(ignore_index=False).reset_index()
vol_surface_long.columns = ['maturity', 'strike', 'price']
vol_surface_long

In [18]:
# Put there the correct values
yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yeilds = np.array([4.93, 4.84, 4.75, 4.65, 4.46, 4.24, 3.99, 3.89, 3.91, 3.97, 4.06, 4.41, 4.34]).astype(float) / 100

In [ ]:
# NSS model calibrate
curve_fit, status = calibrate_nss_ols(yield_maturities, yeilds)
curve_fit

In [ ]:
vol_surface_long['rate'] = vol_surface_long['maturity'].apply(curve_fit)
vol_surface_long

# Apply the Models

In [21]:
def get_last_trade_price(symbol, date, region):
    """Get the last trade price. Requires an active IVol account """
    getMarketData_stocks = ivol.setMethod('/equities/eod/stock-prices')
    marketData_stocks = getMarketData_stocks(symbol=symbol, date=date, region=region)
    return float(marketData_stocks['close'].iloc[0])

In [22]:
def callback(*args):
    """Function used during the minimization process of the models to show time information"""
    global iteration, start, prev, stat
    if iteration == 0:
        print('#iterations\tstep time (s)\ttotal time (s)')
    step = round(time.time() - prev, 3)
    total = round(time.time() - start, 3)
    print(f'{iteration}\t\t{step}\t\t{total}')
    prev = time.time()
    stat.append([iteration, step, total])
    iteration += 1

In [66]:
# General parameters used in the optimization process
method = 'Nelder-Mead'
options = {
    'maxiter': 1e4,
    'disp': True
}
tolerance = 1e-3

# put real data there using the function get_last_trade_price defined above 
last_trade_price = 100 

## Heston Model

In [63]:
# Define variables to be used in optimization
S0 =    last_trade_price # get_last_trade_price(symbol, last_trade_date)
r =     vol_surface_long['rate'].to_numpy('float')
K =     vol_surface_long['strike'].to_numpy('float')
tau =   vol_surface_long['maturity'].to_numpy('float')
P =     vol_surface_long['price'].to_numpy('float')

params = {
    "v0"    : {"x0": 0.1,   "lbub": [1e-3,  0.1]},
    "kappa" : {"x0": 3,     "lbub": [1e-3,  5]},
    "theta" : {"x0": 0.05,  "lbub": [1e-3,  0.1]},
    "sigma" : {"x0": 0.3,   "lbub": [1e-2,  1]},
    "rho"   : {"x0": -0.8,  "lbub": [-1,    0]},
    "lambd" : {"x0": 0.03,  "lbub": [-1,    1]},
}

x0 =    [param["x0"]    for key, param in params.items()]
bnds =  [param["lbub"]  for key, param in params.items()]

In [67]:
# Calibration
def square_error_heston(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    err = np.sum( (P - heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )

    # Zero penalty term - no good guesses for parameters
    pen = 0 # np.sum( [(x_i-x0_i)**2 for x_i, x0_i in zip(x, x0)] )

    return err + pen

start = time.time()
prev = time.time()
iteration = 0
stat = heston_stat = []

result = minimize(square_error_heston, x0, tol=tolerance, method=method, options=options, bounds=bnds, callback=callback)

#iterations	step time (s)	total time (s)
0		5.847		5.847
1		0.753		6.6
2		1.455		8.055
3		0.741		8.796
4		1.453		10.249
5		1.467		11.716
6		0.73		12.446
7		1.025		13.472


KeyboardInterrupt: 

In [ ]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
print(f"v0={v0}, kappa={kappa}, theta={theta}, sigma={sigma}, rho={rho}, lambda={lambd}")

In [ ]:
heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
vol_surface_long['heston_price'] = heston_prices
vol_surface_long

In [ ]:
vol_surface_long['heston_difference'] = vol_surface_long['price'] - vol_surface_long['heston_price']
vol_surface_long['heston_difference'].describe()

## Merton Jump Model

In [33]:
S0 =    last_trade_price  # Replace with real market data
r =     vol_surface_long['rate'].to_numpy(float)
K =     vol_surface_long['strike'].to_numpy(float)
tau =   vol_surface_long['maturity'].to_numpy(float)
P =     vol_surface_long['price'].to_numpy(float)

params = {
    "sigma"     : {"x0": 0.2, "lbub": [1e-2, 1]},
    "lambd"     : {"x0": 0.1, "lbub": [1e-3, 1]},
    "mu_j"      : {"x0": 0,   "lbub": [-1, 1]},
    "sigma_j"   : {"x0": 0.1, "lbub": [1e-2, 1]},
}

x0 =    [param["x0"]    for param in params.values()]
bnds =  [param["lbub"]  for param in params.values()]

In [34]:
def square_error_merton(x):
    sigma, lambd, mu_j, sigma_j = x
    return np.sum((P - merton_price_rec(S0, K, sigma, lambd, mu_j, sigma_j, tau, r))**2 / len(P))

In [ ]:
start = time.time()
prev = time.time()
iteration = 0
stat = merton_stat = []

result = minimize(square_error_merton, x0, tol=tolerance, bounds=bnds, method=method, options=options, callback=callback)

In [ ]:
sigma, lambd, mu_j, sigma_j = result.x
print(f"Calibrated parameters: sigma={sigma}, lambd={lambd}, mu_j={mu_j}, sigma_j={sigma_j}")

# --- Final Volatility Surface and Comparison ---
merton_prices = merton_price_rec(S0, K, sigma, lambd, mu_j, sigma_j, tau, r)
vol_surface_long['merton_price'] = merton_prices
vol_surface_long['merton_difference'] = vol_surface_long['price'] - vol_surface_long['merton_price']
vol_surface_long['merton_difference'].describe()

In [ ]:
vol_surface_long.describe()

## Kou Model

In [39]:
S0 =    last_trade_price  # Replace with real market data
r =     vol_surface_long['rate'].to_numpy(float)
K =     vol_surface_long['strike'].to_numpy(float)
tau =   vol_surface_long['maturity'].to_numpy(float)
P =     vol_surface_long['price'].to_numpy(float)

params = {
    "sigma" : {"x0": 0.2,   "lbub": [1e-2, 1]},
    "lambd" : {"x0": 0.1,   "lbub": [1e-3, 1]},
    "p"     : {"x0": 0.5,   "lbub": [0.01, 0.99]},
    "eta1"  : {"x0": 10,    "lbub": [1, 20]},
    "eta2"  : {"x0": 10,    "lbub": [1, 20]},
}

x0 =    [param["x0"]    for param in params.values()]
bnds =  [param["lbub"]  for param in params.values()]

In [40]:
def square_error_kou(x):
    sigma, lambd, p, eta1, eta2 = x
    return np.sum((P - kou_price_rec(S0, K, sigma, lambd, p, eta1, eta2, tau, r))**2 / len(P))

In [ ]:
iteration = 0
start = time.time()
prev = time.time()
stat = kou_stat = []

result = minimize(square_error_kou, x0, tol=tolerance, bounds=bnds, method=method, options=options, callback=callback)

In [ ]:
sigma, lambd, data_path, eta1, eta2 = result.x
print(f"Calibrated parameters: sigma={sigma}, lambd={lambd}, p={data_path}, eta1={eta1}, eta2={eta2}")

# --- Final Volatility Surface and Comparison ---
kou_prices = kou_price_rec(S0, K, sigma, lambd, data_path, eta1, eta2, tau, r)
vol_surface_long['kou_price'] = kou_prices
vol_surface_long['kou_difference'] = vol_surface_long['price'] - vol_surface_long['kou_price']
vol_surface_long

In [ ]:
vol_surface_long.describe()

## Variance Gamma Model

In [68]:
S0 =    last_trade_price  # Replace with real market data
r =     vol_surface_long['rate'].to_numpy(float)
K =     vol_surface_long['strike'].to_numpy(float)
tau =   vol_surface_long['maturity'].to_numpy(float)
P =     vol_surface_long['price'].to_numpy(float)

params = {
    "sigma"     : {"x0": 0.2, "lbub": [1e-2, 1]},
    "nu"        : {"x0": 0.1, "lbub": [1e-2, 1]}
}

x0 =    [param["x0"]    for param in params.values()]
bnds =  [param["lbub"]  for param in params.values()]

In [69]:
def square_error_vg(x):
    sigma, nu = x
    return np.sum((P - vg_price_rec(S0, K, theta, sigma, nu, tau, r))**2 / len(P))

In [70]:
iteration = 0
start = time.time()
prev = time.time()
stat = vg_stat = []

result = minimize(square_error_vg, x0, tol=tolerance, bounds=bnds, method=method, options=options, callback=callback)

#iterations	step time (s)	total time (s)
0		1.555		1.556
1		0.616		2.172
2		0.59		2.762
3		0.59		3.352
4		0.602		3.954
5		0.288		4.243
6		0.591		4.834
7		0.575		5.41
8		0.574		5.984
9		0.582		6.566
10		0.289		6.855
11		0.577		7.432
12		0.0		7.432
Optimization terminated successfully.
         Current function value: 3403.225769
         Iterations: 13
         Function evaluations: 25


In [ ]:
sigma, nu = result.x
print(f"Calibrated parameters: {sigma=}, {nu=}")

# --- Final Volatility Surface and Comparison ---
vg_prices = vg_price_rec(S0, K, theta, sigma, nu, tau, r)
vol_surface_long['vg_price'] = vg_prices
vol_surface_long['vg_difference'] = vol_surface_long['price'] - vol_surface_long['vg_price']
vol_surface_long

In [ ]:
vol_surface_long.describe()

# Compare the models

In [ ]:
def plot_time_stat(model_name, steps, totals, color):
    plt.plot(steps  , '-',  color=color, label=f'{model_name}-step time (s)')
    plt.plot(totals , '--', color=color, label=f'{model_name}-cumulative time (s)')
    plt.legend()
    plt.xlabel('iteration')
    plt.ylabel('time (s)')

In [ ]:
import matplotlib.colors as mcolors

stats = {
    'heston'    : heston_stat,
    'merton'    : merton_stat,
    'kou'       : kou_stat,
    'vg'        : vg_stat
}

for (model_name, time_stat), color in zip(stats.items(), list(mcolors.TABLEAU_COLORS.keys())[:len(stats)]):
    _, steps, cumulatives = zip(*time_stat)
    plot_time_stat(model_name, steps, cumulatives, color)

In [ ]:
vol_surface_long[['maturity', 'strike', 'heston_difference', 'merton_difference', 'kou_difference', 'vg_difference']] \
    .plot.line(figsize=(10, 6))